<a href="https://colab.research.google.com/github/HendricoYehezky/Tugas-1/blob/main/Bab_4_4_Model_Seleksi%2C_Underfitting%2C_dan_overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install d2l

     |████████████████████████████████| 82 kB 357 kB/s 
     |████████████████████████████████| 20.1 MB 1.7 MB/s 
     |████████████████████████████████| 61 kB 3.8 MB/s 
     |████████████████████████████████| 9.9 MB 39.9 MB/s 
     |████████████████████████████████| 11.6 MB 31.5 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the p

In [12]:
!pip install pyTorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  ERROR: Failed building wheel for pyTorch
  Running setup.py clean for pyTorch
Failed to build pyTorch
    Running setup.py install for pyTorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-u4dd9u6z/pytorch_23df2daef84c462181797c5f65914773/setup.py'"'"'; __file__='"'"'/tmp/pip-install-u4dd9u6z/pytorch_23df2daef84c462181797c5f65914773/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-ww9pvy6v/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/pyTorch Check the logs for full command output.


In [3]:
import math
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [4]:
max_degree = 20  # Maximum degree of the polynomial
n_train, n_test = 100, 100  # Training and test dataset sizes
true_w = np.zeros(max_degree)  # Allocate lots of empty space
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])

features = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(features)
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)  # `gamma(n)` = (n-1)!
# Shape of `labels`: (`n_train` + `n_test`,)
labels = np.dot(poly_features, true_w)
labels += np.random.normal(scale=0.1, size=labels.shape)

In [5]:
# Convert from NumPy ndarrays to tensors
true_w, features, poly_features, labels = [torch.tensor(x, dtype=
    torch.float32) for x in [true_w, features, poly_features, labels]]

features[:2], poly_features[:2, :], labels[:2]

(tensor([[ 0.5521],
         [-0.7752]]),
 tensor([[ 1.0000e+00,  5.5210e-01,  1.5240e-01,  2.8047e-02,  3.8712e-03,
           4.2745e-04,  3.9333e-05,  3.1022e-06,  2.1409e-07,  1.3133e-08,
           7.2507e-10,  3.6391e-11,  1.6743e-12,  7.1105e-14,  2.8041e-15,
           1.0321e-16,  3.5613e-18,  1.1566e-19,  3.5474e-21,  1.0308e-22],
         [ 1.0000e+00, -7.7520e-01,  3.0047e-01, -7.7641e-02,  1.5047e-02,
          -2.3329e-03,  3.0141e-04, -3.3379e-05,  3.2344e-06, -2.7859e-07,
           2.1597e-08, -1.5220e-09,  9.8320e-11, -5.8629e-12,  3.2464e-13,
          -1.6777e-14,  8.1287e-16, -3.7067e-17,  1.5963e-18, -6.5131e-20]]),
 tensor([5.3308, 2.7229]))

In [7]:
def evaluate_loss(net, data_iter, loss):  
    """Evaluate the loss of a model on the given dataset."""
    metric = d2l.Accumulator(2)  # Sum of losses, no. of examples
    for X, y in data_iter:
        out = net(X)
        y = y.reshape(out.shape)
        l = loss(out, y)
        metric.add(l.sum(), l.numel())
    return metric[0] / metric[1]

In [8]:
def train(train_features, test_features, train_labels, test_labels,
          num_epochs=400):
    loss = nn.MSELoss(reduction='none')
    input_shape = train_features.shape[-1]
    # Switch off the bias since we already catered for it in the polynomial
    # features
    net = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
    batch_size = min(10, train_labels.shape[0])
    train_iter = d2l.load_array((train_features, train_labels.reshape(-1,1)),
                                batch_size)
    test_iter = d2l.load_array((test_features, test_labels.reshape(-1,1)),
                               batch_size, is_train=False)
    trainer = torch.optim.SGD(net.parameters(), lr=0.01)
    animator = d2l.Animator(xlabel='epoch', ylabel='loss', yscale='log',
                            xlim=[1, num_epochs], ylim=[1e-3, 1e2],
                            legend=['train', 'test'])
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(epoch + 1, (evaluate_loss(net, train_iter, loss),
                                     evaluate_loss(net, test_iter, loss)))
    print('weight:', net[0].weight.data.numpy())

In [13]:
### Fitting Polinomial Orde Tiga ###
# Pick the first four dimensions, i.e., 1, x, x^2/2!, x^3/3! from the
# polynomial features
train(poly_features[:n_train, :4], poly_features[n_train:, :4],
      labels[:n_train], labels[n_train:])

weight: [[ 5.0119324  1.2265548 -3.4157116  5.567762 ]]


ImportError: ignored

<Figure size 252x180 with 1 Axes>

In [14]:
### Fitting Fungsi Linear (Overfitting) ###
# Pick the first two dimensions, i.e., 1, x, from the polynomial features
train(poly_features[:n_train, :2], poly_features[n_train:, :2],
      labels[:n_train], labels[n_train:])

weight: [[3.522307 4.156145]]


ImportError: ignored

<Figure size 252x180 with 1 Axes>

In [ ]:
### Fitting Fungsi Polinomial Derajat Tinggi (Overfitting) ###
# Pick all the dimensions from the polynomial features
train(poly_features[:n_train, :], poly_features[n_train:, :],
      labels[:n_train], labels[n_train:], num_epochs=1500)

ImportError: ignored

<Figure size 252x180 with 1 Axes>